In [174]:
import json
import pandas as pd

with open('categories_label.json', 'r') as f:
    categories_label = json.load(f)

In [175]:
categories_label

{'Germany': {'continent': 'Europe', 'activities': ['Restaurant']},
 'France': {'continent': 'Europe', 'activities': ['Restaurant']},
 'Italy': {'continent': 'Europe', 'activities': ['Restaurant']},
 'Spain': {'continent': 'Europe', 'activities': ['Restaurant']},
 'Portugal': {'continent': 'Europe', 'activities': ['no label']},
 'Netherlands': {'continent': 'Europe', 'activities': ['Restaurant']},
 'Belgium': {'continent': 'Europe',
  'activities': ['Restaurant', 'Plane', 'Gym']},
 'Switzerland': {'continent': 'Europe', 'activities': ['Restaurant', 'sport']},
 'Brazil': {'continent': 'South America',
  'activities': ['Restaurant', 'Sao Paulo']},
 'Argentina': {'continent': 'South America',
  'activities': ['Restaurant', 'sport', 'Cinema']},
 'Colombia': {'continent': 'South America',
  'activities': ['Restaurant', 'sport']},
 'Chile': {'continent': 'South America',
  'activities': ['Restaurant', 'sport']},
 'Peru': {'continent': 'South America', 'activities': ['Restaurant', 'sport']}}

In [176]:
all_activities = set()
for country_info in categories_label.values():
    all_activities.update(country_info["activities"])
all_activities = sorted(all_activities)

all_activities

['Cinema', 'Gym', 'Plane', 'Restaurant', 'Sao Paulo', 'no label', 'sport']

In [177]:
data = {"Country": []}
for activity in all_activities:
    data[activity] = []

for country, info in categories_label.items():
    row = {activity: True if activity in info["activities"] else False for activity in all_activities}
    row["Country"] = country
    data["Country"].append(country)
    for activity in all_activities:
        data[activity].append(row[activity])

df = pd.DataFrame(data)
df

,Country,Cinema,Gym,Plane,Restaurant,Sao Paulo,no label,sport
0,Germany,False,False,False,True,False,False,False
1,France,False,False,False,True,False,False,False
2,Italy,False,False,False,True,False,False,False
3,Spain,False,False,False,True,False,False,False
4,Portugal,False,False,False,False,False,True,False
5,Netherlands,False,False,False,True,False,False,False
6,Belgium,False,True,True,True,False,False,False
7,Switzerland,False,False,False,True,False,False,True
8,Brazil,False,False,False,True,True,False,False
9,Argentina,True,False,False,True,False,False,True


In [184]:
conn = duckdb.connect('activities.db')

# Create the activities_table if it doesn't exist using BOOLEAN instead of INTEGER
create_table_query = '''
CREATE TABLE IF NOT EXISTS activities_table1(
    Country VARCHAR,
    {},
    timestamp_date TIMESTAMP,
    PRIMARY KEY (Country, timestamp_date)
)
'''.format(", ".join([f'"{activity}" BOOLEAN' for activity in all_activities]))

# Print the CREATE TABLE statement for debugging
print(f"Executing: {create_table_query}")
conn.execute(create_table_query)
print(f"Executing2: {create_table_query}")


Executing: 
CREATE TABLE IF NOT EXISTS activities_table1(
    Country VARCHAR,
    "Cinema" BOOLEAN, "Gym" BOOLEAN, "Plane" BOOLEAN, "Restaurant" BOOLEAN, "Sao Paulo" BOOLEAN, "no label" BOOLEAN, "sport" BOOLEAN,
    timestamp_date TIMESTAMP,
    PRIMARY KEY (Country, timestamp_date)
)

Executing2: 
CREATE TABLE IF NOT EXISTS activities_table1(
    Country VARCHAR,
    "Cinema" BOOLEAN, "Gym" BOOLEAN, "Plane" BOOLEAN, "Restaurant" BOOLEAN, "Sao Paulo" BOOLEAN, "no label" BOOLEAN, "sport" BOOLEAN,
    timestamp_date TIMESTAMP,
    PRIMARY KEY (Country, timestamp_date)
)



In [183]:
import duckdb
conn = duckdb.connect('activities.db')

In [185]:


# Step 2: Execute a query to read data from the activities_table
query = "SELECT * FROM activities_table1"

# Step 3: Fetch the results
# You can fetch results directly into a pandas DataFrame or a list of tuples
df = conn.execute(query).fetchdf()  # This will return a DataFrame

# Step 4: Print the DataFrame
print(df)

Empty DataFrame
Columns: [Country, Cinema, Gym, Plane, Restaurant, Sao Paulo, no label, sport, timestamp_date]
Index: []


In [189]:
print(df)

Empty DataFrame
Columns: [Country, Cinema, Gym, Plane, Restaurant, Sao Paulo, no label, sport, timestamp_date]
Index: []


In [188]:
insert_data = []
for _, row in df.iterrows():
    country = row["Country"].replace("'", "''")  # Escape single quotes
    print(country)
    activities = tuple(bool(row[activity]) for activity in all_activities)  # Convert activities to booleans
    current_timestamp = pd.Timestamp.now()  # Get current timestamp
    print(current_timestamp)
    # Prepare the data to insert: include country, activities, and timestamp
    data_row = (country,) + activities + (current_timestamp,)
    print("here", data_row)
    

In [187]:
insert_data

[]

In [172]:
insert_data = []
for _, row in df.iterrows():
    country = row["Country"].replace("'", "''")  # Escape single quotes
    print(country)
    activities = tuple(bool(row[activity]) for activity in all_activities)  # Convert activities to booleans
    current_timestamp = pd.Timestamp.now()  # Get current timestamp
    print(current_timestamp)
    # Prepare the data to insert: include country, activities, and timestamp
    data_row = (country,) + activities + (current_timestamp,)
    print("here", data_row)
    
    # Check if the last entry for this country is different
    query = f'SELECT * FROM activities_table WHERE Country = ? ORDER BY timestamp DESC LIMIT 1'
    last_entry = conn.execute(query, (country,)).fetchone()
    print("last_entry")

    # Check if the last entry exists and if there is a change
    if last_entry:
        # Compare activities to see if there's a change
        if last_entry[1:-1] != activities:  # Exclude Country and timestamp for comparison
            insert_data.append(data_row)
    else:
        # If no previous entry, add the current data
        insert_data.append(data_row)


Germany
2024-10-01 15:59:26.786870
here ('Germany', False, False, False, True, False, False, False, Timestamp('2024-10-01 15:59:26.786870'))


BinderException: Binder Error: Referenced column "timestamp" not found in FROM clause!
Candidate bindings: "activities_table.Country"
LINE 1: ...ities_table WHERE Country = ? ORDER BY timestamp DESC LIMIT 1
                                                  ^

In [156]:
insert_data

[]

In [157]:
activities

(False, False, False, True, False, False, True)

In [121]:
data_row

NameError: name 'data_row' is not defined

In [195]:


# Step 2: Execute a query to read data from the activities_table
query = "SELECT * FROM activities_table"

# Step 3: Fetch the results
# You can fetch results directly into a pandas DataFrame or a list of tuples
df = conn.execute(query).fetchdf()  # This will return a DataFrame

# Step 4: Print the DataFrame
print(df)

Empty DataFrame
Columns: [Country]
Index: []


In [197]:
import duckdb

# Path to your .db file
db_file = 'activities.db'

# Connect to the DuckDB database
connection = duckdb.connect(database=db_file)

# Example: Query to fetch all records from a table (replace 'your_table_name' with your actual table name)
query = 'SELECT * FROM activities_table'

# Execute the query and fetch the results into a DataFrame
df = connection.execute(query).fetchdf()

# Print the DataFrame
print(df)

# Close the connection
connection.close()

Empty DataFrame
Columns: [Country]
Index: []


In [200]:
import sqlite3

# Path to your .db file
db_file = 'activitiessql.db'

# Connect to the SQLite database
connection = sqlite3.connect(db_file)

# Create a cursor object to interact with the database
cursor = connection.cursor()

# Example: Query to fetch all records from a table (replace 'your_table_name' with your actual table name)
query = 'SELECT * FROM activities_table'

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Iterate through the results and print them
for row in results:
    print(row)

# Close the cursor and connection
cursor.close()
connection.close()


('Germany', 0, 0, 0, 0, 0, 0, 0, '2024-10-01T16:13:44.942545')
('France', 0, 0, 0, 0, 0, 0, 0, '2024-10-01T16:13:44.942845')
('Italy', 0, 0, 0, 1, 0, 0, 0, '2024-10-01T16:13:44.943064')
('Spain', 0, 0, 0, 1, 0, 0, 0, '2024-10-01T16:13:44.943328')
('Portugal', 0, 0, 0, 0, 0, 1, 0, '2024-10-01T16:13:44.943534')
('Netherlands', 0, 0, 0, 1, 0, 0, 0, '2024-10-01T16:13:44.943736')
('Belgium', 0, 1, 1, 1, 0, 0, 0, '2024-10-01T16:13:44.943928')
('Switzerland', 0, 0, 0, 1, 0, 0, 1, '2024-10-01T16:13:44.944118')
('Brazil', 0, 0, 0, 1, 1, 0, 0, '2024-10-01T16:13:44.944331')
('Argentina', 1, 0, 0, 1, 0, 0, 1, '2024-10-01T16:13:44.944521')
('Colombia', 0, 0, 0, 1, 0, 0, 1, '2024-10-01T16:13:44.944707')
('Chile', 0, 0, 0, 1, 0, 0, 1, '2024-10-01T16:13:44.944894')
('Peru', 0, 0, 0, 1, 0, 0, 1, '2024-10-01T16:13:44.945079')
('Germany', 1, 1, 1, 1, 1, 0, 0, '2024-10-01T16:14:10.004654')


In [ ]:
asset_folder = Path(os.environ["domino_dataset_dir"] + "/local" + os.environ["folder_deus"]  + "folder_march")